<a href="https://colab.research.google.com/github/samruddhideode/Sanvadita/blob/main/sanvadita_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fastText

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.1 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fastText: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3161998 sha256=b0463615319c48c346d6722ef445c94399c433625e1565cd0aca2f712e051850
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fastText


In [3]:
import fasttext.util

In [12]:
import sklearn
import pandas as pd
import string
import re
import numpy as np

from sklearn.naive_bayes import  MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.preprocessing import MinMaxScaler

In [5]:
fasttext.util.download_model('mr',if_exists = 'ignore')

'cc.mr.300.bin'

In [6]:
 ft = fasttext.load_model('cc.mr.300.bin')

In [7]:
train_set = pd.read_csv("https://raw.githubusercontent.com/l3cube-pune/MarathiNLP/main/L3CubeMahaSent%20Dataset/tweets-train.csv")
test_set = pd.read_csv("https://raw.githubusercontent.com/l3cube-pune/MarathiNLP/main/L3CubeMahaSent%20Dataset/tweets-test.csv")
valid_set= pd.read_csv("https://raw.githubusercontent.com/l3cube-pune/MarathiNLP/main/L3CubeMahaSent%20Dataset/tweets-valid.csv")

In [ ]:
print(train_set.head())
print(test_set.head())
print(valid_set.head())

In [9]:
print(len(train_set.tweet)) #12114 tweets in the dataset
print(len(train_set.label)) #12114 labels in the dataset

12114
12114


In [10]:
def preprocess_tweet(tweet):
    tweet = re.sub('@[^\s]+','',tweet) #removing usernames, handles
    tweet= re.sub('#\w*[a-zA-Z]+\w*', '',tweet) # removing hashtags
    tweet= re.sub('\d+', '', tweet) # Removing numbers
    tweet= re.sub('[a-zA-Z0-9]+', '',tweet) # removing english words
    tweet= re.sub('[^\w\s]+', '',tweet) # removing punctuations
    return tweet

train_set.tweet = train_set.tweet.apply(preprocess_tweet)
test_set.tweet = test_set.tweet.apply(preprocess_tweet)
valid_set.tweet = valid_set.tweet.apply(preprocess_tweet)

Embedding using facebook fasttext

In [11]:
# average of all fasttext embeddings in a tweet

def vectorize(tweets):
  i = 0
  tweet_vectors = []
  for tweet in tweets:
    sentence=[]
    for word in tweet:
      sentence.append(ft.get_word_vector(word))
    tweet_vectors.append(np.mean(sentence, axis = 0))
  return tweet_vectors

tweet_vectors_train = vectorize(train_set.tweet)
tweet_vectors_test = vectorize(test_set.tweet)
tweet_vectors_valid = vectorize(valid_set.tweet)



1. Knn Classifier using fasttext facebook

In [18]:
k_range = range(1,100)
roc_auc = []
accuracy = []
for k in k_range:
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(tweet_vectors_train,train_set.label)
  roc_auc.append(roc_auc_score(test_set.label,knn.predict_proba(tweet_vectors_test),multi_class='ovr'))
  accuracy.append(accuracy_score(test_set.label, knn.predict(tweet_vectors_test)))

In [24]:
k = accuracy.index(max(accuracy))+1
print('k: ',k)

k:  53


Training knn with k = 53 using facebook fasttext

In [25]:
knn = KNeighborsClassifier(n_neighbors=53)
knn.fit(tweet_vectors_train,train_set.label)

train_roc_auc = roc_auc_score(train_set.label,knn.predict_proba(tweet_vectors_train),multi_class='ovr')
train_accuracy = accuracy_score(train_set.label, knn.predict(tweet_vectors_train))

test_roc_auc = roc_auc_score(test_set.label,knn.predict_proba(tweet_vectors_test),multi_class='ovr')
test_accuracy = accuracy_score(test_set.label, knn.predict(tweet_vectors_test))

valid_roc_auc = roc_auc_score(valid_set.label,knn.predict_proba(tweet_vectors_valid),multi_class='ovr')
valid_accuracy = accuracy_score(valid_set.label, knn.predict(tweet_vectors_valid))

In [26]:
print("knn results with facebook fasttext: ")
print("train roc_auc: ",train_roc_auc)
print("test roc_auc: ", test_roc_auc)
print("valid roc_auc: ", valid_roc_auc)

print("\n")
print("train accuracy: ",train_accuracy)
print("test accuracy: ", test_accuracy)
print("valid accuracy: ", valid_accuracy)

knn results with facebook fasttext: 
train roc_auc:  0.769034680482584
test roc_auc:  0.7376278518518519
valid roc_auc:  0.7508233333333334


train accuracy:  0.5728908700676902
test accuracy:  0.5528888888888889
valid accuracy:  0.5526666666666666


ValueError: ignored